In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import os

import time
import csv
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image
import plotly.graph_objects as go

In [2]:
training_dir = "D:/Datasets/Fish/processed/train/New folder/"
validation_dir = "D:/Datasets/Fish/processed/validation/New folder/"
test_dir = "D:/Datasets/Fish/processed/test/New folder/"

def count_files(path):
    Number_Of_Files=0
    for file in os.walk(path):
    #     print(file)
        for files in file[2]:
            Number_Of_Files=Number_Of_Files+1
    return Number_Of_Files

count_files(training_dir)

44956

In [119]:
l = 90 # clip length, the number of frames in the video segment fed to the ConvLSTM
h = 10 # look-ahead. we average the consecutive frame difference over the h+1 next frames, including the current
# split computation into two loops. first loop computes y_tilde (i.e. inter-frame differences). 2nd loop averages y_tildes in next h frames
# this function assumes the moving objects in the frame have a pixel intensity of 1 and the background is 0
def compute_targets(image_folder):
    T = count_files(image_folder)
    start_frame = l-1 # first frame inded in the valid range
    end_frame = T-2-h # last frame index in the valid range, assuming images start at t=0 and go to t=T-1
    y_tilde = np.zeros(T)
    for t in range(T-1): #T-1
        pil_img_t0 = load_img(image_folder+f'{t:05}' + '.png', color_mode='grayscale', target_size=None)
        pil_img_t1 = load_img(image_folder+f'{t+1:05}' + '.png', color_mode='grayscale', target_size=None)
        
        img_array_0 = img_to_array(pil_img_t0, data_format='channels_last', dtype='uint8')
        img_array_1 = img_to_array(pil_img_t1, data_format='channels_last', dtype='uint8')
        #we must convert to float32 before performing subtraction, otherwise all negative values will be set as 255
        img_array_0 = img_array_0.astype(np.float32)/255
        img_array_1 = img_array_1.astype(np.float32)/255
        
        y_tilde[t] = np.sum(np.abs(img_array_1 - img_array_0))/np.prod(img_array_0.shape)#Also must normalize by numbr of pixels (later) using the target size in load_img
        
#         print(np.prod(img_array_0.shape))
#         pil_im = Image.fromarray(np.abs(img_array_1 - img_array_0)[:,:,0], mode='F') # Mode=F allows direct conversion from 32bit pixels. mode L is for uint8
#         pil_im.save(r'C:\Users\MrLin\Documents\Experiments\ConvLSTM_forcasting\Results\y_tilde\img_' + f'{t:06}' + '.tif')
    
    # label each frame algorithmically
    b=0.1
    c=0.001
    y_label = np.zeros(T)
    for t in range(start_frame, end_frame+1):
        if np.ptp(y_tilde[t:t+h+1]) >= c and y_tilde[t]<=(np.min(y_tilde[t:t+h+1])+b*np.ptp(y_tilde[t:t+h+1])):
            y_label[t] = 1
        
    return y_tilde, y_label
        


In [ ]:
# baseline/'naive' classifier algorithm to compare with convLSTM. Can also try regular LSTM.
# the algorithm has access to the same l prior frames (only y_tilde data) and must make a prediction on the next h+1 frames
# The algorithm tries to exploit short-term correlations in the signal by assuming recent spikes will preceed future spikes

l = 90 # clip length, the number of frames in the video segment fed to the ConvLSTM
h = 10 # look-ahead. we average the consecutive frame difference over the h+1 next frames, including the current
d = 45 # baseline alg will look-back this many frames to make the prediction
def baseline_prediction(y_tilde):
    T = len(y_tilde)
    start_frame = l-1 # first frame inded in the valid range
    end_frame = T-2-h # last frame index in the valid range, assuming images start at t=0 and go to t=T-1
    y_label_baseline = np.zeros(T)
    b=0.1
    c=0.001

    for t in range(start_frame, end_frame+1):
        if np.ptp(y_tilde[t-d:t]) >= c and y_tilde[t-1]<=(np.min(y_tilde[t-d:t])+b*np.ptp(y_tilde[t-d:t])):
            y_label_baseline[t] = 1
    return y_label_baseline

# Y_label_baseline = baseline_prediction(Y_tilde)

In [9]:
# baseline predictor outputting probabilities of positive
l = 90 # clip length, the number of frames in the video segment fed to the ConvLSTM
h = 10 # look-ahead. we average the consecutive frame difference over the h+1 next frames, including the current
d = 89 # baseline alg will look-back this many frames to make the prediction
def baseline_prediction_probabilistic(y_tilde):
    T = len(y_tilde)
    start_frame = l-1 # first frame inded in the valid range
    end_frame = T-2-h # last frame index in the valid range, assuming images start at t=0 and go to t=T-1
    y_label_baseline = np.zeros(T)
    b=0.2
    c=0.0006 # approx mean range of y_tilde across d time-steps
    gamma1 = 5300
    gamma2 = 11000
    center1 = c
    
    P1 = lambda x, c, gamma1: 1/(1 + np.exp(-gamma1*(x-c))) # input x is the range np.ptp(y_tilde[t-d:t]). higher range--> higher probability
    P2 = lambda x, low_point, gamma2: 1/(1 + np.exp(gamma2*(x-low_point))) # input x is y tilde at t-1. lower x-->higher probability
    
    for t in range(start_frame, end_frame+1):
        center2 = np.min(y_tilde[t-d:t])+b*np.ptp(y_tilde[t-d:t])
        y_label_baseline[t] = (P1(np.ptp(y_tilde[t-d:t]), center1, gamma1) + P2(y_tilde[t-1], center2, gamma2))/2
    return y_label_baseline

In [12]:
# Compute the targets and save
# Y_tilde, Y_label = compute_targets(training_dir)
# Y_tilde, Y_label = compute_targets(validation_dir)
# Y_tilde, Y_label = compute_targets(test_dir)

# np.save(r'D:\Datasets\Fish\processed\targets\validation\Y_tilde', Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\validation\Y_label', Y_label)
# np.save(r'D:\Datasets\Fish\processed\targets\train\Y_tilde', Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\train\Y_label', Y_label)
# np.save(r'D:\Datasets\Fish\processed\targets\test\Y_tilde', Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\test\Y_label', Y_label)

#make baseline predictions and save
# Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\validation\Y_tilde.npy')
# Y_label_baseline = baseline_prediction(Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\validation\Y_label_baseline', Y_label_baseline)

Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_tilde.npy')
Y_pred_baseline = baseline_prediction_probabilistic(Y_tilde)
Y_pred_baseline
np.save(r'D:\Datasets\Fish\processed\targets\test\Y_pred_baseline', Y_pred_baseline)

# Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\train\Y_tilde.npy')
# Y_label_baseline = baseline_prediction(Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\train\Y_label_baseline', Y_label_baseline)

In [23]:
# check variable shapes
Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_tilde.npy')
Y_label = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_label.npy')
Y_pred_baseline = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_pred_baseline.npy')
print("Y_tilde: {}, Y_label: {}, y_pred_baseline: {}".format(Y_tilde.shape[0], Y_label.shape[0], Y_pred_baseline.shape[0]))

Y_tilde: 5367, Y_label: 5367, y_pred_baseline: 5367


In [31]:
# Plot y tilde data over frames
# Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\validation\Y_tilde.npy')
# Y_label = np.load(r'D:\Datasets\Fish\processed\targets\validation\Y_label.npy')
Y_tilde.shape
x = np.arange(0, len(Y_label), 1)


# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(name='positive',
                    x=x[1:], y=Y_tilde[1:-1],
                    mode='lines',
                    
                    line=dict(
                        width=1,
                        color='gray'
                    ),
                    showlegend=False

))

fig.add_trace(go.Scatter(name='negative (ground truth)',
                    x=x[Y_label==0][1:], y=Y_tilde[Y_label==0][1:-1],
                    mode='markers',
                    
                    marker=dict(
                        size=5,
                        color='black'
                    ),

))

threshold = 0.95
fig.add_trace(go.Scatter(name='positive (baseline prediction)',
                    x=x[Y_pred_baseline>threshold][1:], y=Y_tilde[Y_pred_baseline>threshold][1:-1],
                    mode='markers',
                    
                    marker=dict(
                        color='green',
                        size=10,
                        opacity=0.6,
                        line=dict(
                            color='black',
                            width=1
                        )
                    ),

))

fig.add_trace(go.Scatter(name='positive (ground truth)',
                    x=x[Y_label==1][1:], y=Y_tilde[Y_label==1][1:-1],
                    mode='markers',
                    
                    marker=dict(
                        size=5,
                        color='hotpink'
                    ),

))

fig.update_layout(
    width=900,
    height=700,
                
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01,
        font_size=14
    ),
    xaxis=dict(title_text='frames', title_font_size=18),
    yaxis=dict(title_text='\u1EF9', title_font_size=18)

)

fig.show()


In [41]:
Y_acc = np.diff(Y_tilde, n=1, axis=-1)
colsp = np.linspace(0,100,100)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(name='Y_acc',
                    x=Y_tilde[700:800], y=Y_acc[700:800],
                    mode='lines+markers',
                    
                    marker=dict(
                        size=6,
                        color=colsp
                    )    
))

fig2.show()

In [42]:
fig = go.Figure(data=[go.Histogram(x=Y_tilde, cumulative_enabled=True, histnorm='probability')])

fig.show()